To do:
- ADD WEIGHTED LOSS FUNCTION FOR HITS ON 1 FOR POSITIVE LABELLING AS THIS IS WAY MORE RARE = HIGHER "REWARD"
- Have a look at learning rate and gradient norm clipping which I need to read up on.
    - In addition to this, look at the implications of gradient accumulation steps
    - Much of this pipeline was constrained due to computational restrictions which I think was caused by errors and not actual training process.
- Early stopping: load_best_model_at_end=True
- Split into validation set as well (80:10:10)
- Hyperparameter tuning (Alpha, learning rate, batch size so on - not sure how to figure this out)
    - There is precedence for no hyperparameter tuning from the author of the OG NLI model that DEBATE is based on = Due to computational restrains and the points from this paper, no hyperparameter tuning was performed in this case. The model tuning in itself is also not the primary focus in this paper, but simply serves as a tool for the actual inquiry into blame in the Danish Parliament
- Context / No context?
    - Most likely no context, as we do not have the academic reasons for doing so 
    - Model actually also seems to perform better without (f1 of .66 for no context on random test set vs. . 55 on gold labelled for context model) This might also make sense as the context is not taken into account in the DEBATE model


In [ ]:
%pip install -r "requirements_bert.txt"

In [1]:
import torch
import transformers
from peft import LoraConfig, get_peft_model, TaskType
import bitsandbytes
import accelerate
import datasets
#import scikit-learn
import numpy as np
import pandas as pd
from datetime import datetime
from transformers import Conv1D, AutoModelForSequenceClassification, AutoTokenizer, TrainingArguments, Trainer, BitsAndBytesConfig, AutoModelForCausalLM
from sklearn.model_selection import train_test_split
from datasets import Dataset

/work/RuneEgeskovTrust#9638/miniconda3/envs/blame/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_name = "jhu-clsp/mmBERT-base"

quantization_config = BitsAndBytesConfig(
                                        load_in_4bit=True,
                                         bnb_4bit_compute_dtype=torch.bfloat16,
                                         bnb_4bit_quant_type="nf4",
                                         bnb_4bit_use_double_quant=True,
                                         )

model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    device_map="auto",
    quantization_config=quantization_config,
)

tokenizer = AutoTokenizer.from_pretrained(model_name)

Some weights of ModernBertForSequenceClassification were not initialized from the model checkpoint at jhu-clsp/mmBERT-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
lora_config = LoraConfig(
    r=8,  # Low-rank dimension
    lora_alpha=16,
    lora_dropout=0.05,
    target_modules=["Wqkv"],  # Fine-tuning the attention layer specifically
)

lora_model = get_peft_model(model, lora_config)
lora_model.print_trainable_parameters()

trainable params: 540,672 || all params: 308,072,450 || trainable%: 0.1755


In [4]:
'''
Consider the batch size, could be increased for efficiency purposes.
ADD WEIGHTED LOSS FUNCTION FOR HITS ON 1 FOR POSITIVE LABELLING AS THIS IS WAY MORE RARE = HIGHER "REWARD"
Have a look at learning rate and gradient norm clipping which I need to read up on.
    In addition to this, look at the implications of gradient accumulation steps
    Much of this pipeline was constrained due to computational restrictions which I think was caused by errors and not actual training process.
Early stopping: load_best_model_at_end=True
'''

training_args = TrainingArguments(
    output_dir='./results',
    learning_rate=8e-4, # Learning rate copied from mmBERT paper as they found this to perform best
    num_train_epochs=3,
    per_device_train_batch_size=8,
    auto_find_batch_size=True, # Allows for auto adjusting of batch to avoid OOM
    gradient_accumulation_steps=12,  # Simulate larger batch size

    logging_steps=1,
    eval_strategy="epoch",
    save_strategy="epoch",
    fp16=True,  # Enable mixed precision
    dataloader_pin_memory=False,
    remove_unused_columns=True, # Avoiding manual handling of residual text columns
    max_grad_norm=1.0,

    disable_tqdm=False,
)

In [5]:

dataframe_12345 = pd.read_json("/work/RuneEgeskovTrust#9638/Bachelor/training_data/subset_1_2_3_4_5_cleaned_training_data.json")

val_dataframe = pd.read_json("/work/RuneEgeskovTrust#9638/Bachelor/Bachelor_project/Model_data/validation_set.json")


In [6]:
# From dataframe to dataset for mapping tokenizer function 

def tokenize_function(example):
    return tokenizer(example["text"], padding="max_length", truncation=True)

In [7]:
val_dataframe = val_dataframe[['text', 'label']]

dataframe_12345 = dataframe_12345[['text', 'label']]

val_dataset = Dataset.from_pandas(val_dataframe)

dataset = Dataset.from_pandas(dataframe_12345)

tokenized_val = val_dataset.map(tokenize_function)

tokenized_dataset = dataset.map(tokenize_function)

Map: 100%|██████████| 3600/3600 [00:06<00:00, 539.46 examples/s]


In [8]:
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {
        'accuracy': accuracy_score(labels, predictions),
        'f1': f1_score(labels, predictions, average='macro') # Macro is better suited for imbalanced data
    }

In [ ]:
'''
This has still not been tested, but will be implemented soon
'''

from collections import Counter

labels = random_dataset['label'].tolist()
class_counts = Counter(labels)
total = sum(class_counts.values())

# Higher weight = more emphasis
weights = [total/class_counts[0], total/class_counts[1]]
class_weights = torch.tensor(weights, dtype=torch.float)

#define custom trainer that uses weigted loss
import torch.nn as nn

class WeightedTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
        outputs = model(**inputs)
        logits = outputs.get("logits")
        
        # Define weighted loss
        loss_fct = nn.CrossEntropyLoss(weight=class_weights.to(model.device))
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        
        return (loss, outputs) if return_outputs else loss

In [11]:
tokenized_dataset["label"]

Column([0, 0, 1, 0, 1])

In [ ]:
trainer = Trainer(
    model=lora_model,
    args=training_args,
    train_dataset=tokenized_dataset,
    eval_dataset=tokenized_val,#split_dataset["test"],
    compute_metrics=compute_metrics,
)


trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,2.520200,0.595228,0.721591,0.712677
2,2.620200,0.474798,0.789773,0.781117


In [ ]:
eval_results = trainer.evaluate()
print(eval_results)
with open("/work/RuneEgeskovTrust#9638/Bachelor/Bachelor_project/Evalresult12345templates.txt", "w") as f:
    f.write(str(eval_results))

# This is where we should very much remember to save the finetuned model locally as this contains the new weights for use in analyzing new text
lora_model.save_pretrained(f"output/mmBERT/template_1_2_3_4_5_model")

{'eval_loss': 0.6536552906036377, 'eval_accuracy': 0.6136363636363636, 'eval_f1': 0.525079365079365, 'eval_runtime': 30.8398, 'eval_samples_per_second': 5.707, 'eval_steps_per_second': 0.713, 'epoch': 3.0}


In [ ]:
'''
We would expect to see a gradual decrease in both training and validation loss.
If either om them split too far from eachother that indicates issues with the training process.
The process itself should be pretty smooth with no dips either up or down.
'''